In [188]:
import configparser
import paramiko

In [189]:
# file_path = '/opt/mytimes-props/prop/mytimes.properties'
file_path = '/opt/batchMyTimes/appConfig.properties'
# file_path = '/opt/backup_props/appConfig.properties_backup_19July'

server_1 = '192.168.36.98'
server_2 = '172.24.81.47'
# server_2 = '192.168.43.239'

In [190]:
DUMMY_SECTION_NAME = 'dummy_section'

def get_generic_config(prop_content, section_name = DUMMY_SECTION_NAME):
    """
    prop_content - file content of the property file
    """
    config_string = '[' + section_name + ']\n' + prop_content
    config = configparser.RawConfigParser()
    config.optionxform = str
    config.read_string(config_string)
    return config

def get_dict_config(config, section_name = DUMMY_SECTION_NAME):
    return dict(config._sections[section_name])

In [191]:
def get_remote_file_content(server_ip, file_path, username, password):
    with paramiko.SSHClient() as ssh_client:
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(hostname=server_ip,username=username ,password=password)
        with ssh_client.open_sftp() as sftp_client:
            with sftp_client.open(file_path) as remote_file:
                content = remote_file.read()
                content = content.decode('utf-8')
    return content

In [192]:
ssh_config_file_loc = '/Users/yash.dalmia/spam_filter/jupyter_notebooks/config/config.ini'

config_ssh = configparser.ConfigParser()
config_ssh.read(ssh_config_file_loc)

['/Users/yash.dalmia/spam_filter/jupyter_notebooks/config/config.ini']

In [193]:
username=config_ssh['LOGIN']['username']
password=config_ssh['LOGIN']['password']

def get_config_dict_server(server_ip, file_path = file_path, username = username, password = password):
    file_content = get_remote_file_content(server_ip, file_path, username, password)
    config_file = get_generic_config(file_content)
    dict_config = get_dict_config(config_file)
    return dict_config

In [194]:
dict_config_1 = get_config_dict_server(server_1)
dict_config_2 = get_config_dict_server(server_2)
set_1 = set(dict_config_1.items())
set_2 = set(dict_config_2.items())

In [195]:
print('server 1: {}\nserver 2: {}'.format(server_1, server_2))

server 1: 192.168.36.98
server 2: 172.24.81.47


In [196]:
##those in 1 that are different or non-existent in 2
# print(set_1 - set_2)

In [197]:
##those in 2 that are different or non-existent in 1
# print(set_2 - set_1)

In [198]:
key_set_diff_1_2 = {x[0] for x in (set_1-set_2)}
key_set_diff_2_1 = {x[0] for x in (set_2-set_1)}

In [199]:
##printing all the differnces between the property file of the two servers in a beautified format 
print('{:50}   {:^30} : {:^30}\n'.format('', server_1, server_2))

#iterates over those keys which are present on both the servers
for common_key in (key_set_diff_1_2 & key_set_diff_2_1):
    print('{:50} : {:^30} : {:^30}'.format(common_key, dict_config_1[common_key], dict_config_2[common_key]))

#iterates over only those keys that are present on server 1 and not on server 2
for key in (key_set_diff_1_2 - key_set_diff_2_1):
    print('{:50} : {:^30} : {:^30}'.format(key, dict_config_1[key], ''))

#iterates over only those keys that are present on server 2 and not on server 1
for key in (key_set_diff_2_1 - key_set_diff_1_2):
    print('{:50} : {:^30} : {:^30}'.format(key, '', dict_config_2[key]))

                                                             192.168.36.98          :          172.24.81.47         

pastCmtsBatch.enable                               :             false              :              true             
mytimes.profanityFilter.nonLiveRun                 :              true              :                               
